## 05 – NN-Energieberechnung (ANI2x) für Polymerfragmente

Dieses Notebook nutzt ein neuronales Netz (TorchANI, z. B. ANI2x) für Single-Point-Energien von aus Monomeren aufgebauten Fragmenten. Die Polymer-Konfigurationen werden mit `iterative_extend_smiles` erzeugt und die Berechnungen parallel (ähnlich zu `opt_helpers.py`) ausgeführt.

Referenz: Sammlung relevanter NN-Modelle in der Chemie: [Neural-Network-Models-for-Chemistry](https://github.com/Eipgen/Neural-Network-Models-for-Chemistry)

Hinweise:
- Es wird TorchANI verwendet; Installation erfolgt unten im Notebook falls erforderlich.

In [5]:
import importlib
import os
import shutil
import sys

# Install TorchANI using uv if missing
try:
    import torchani  # noqa: F401
    print("TorchANI already installed.")
except Exception:
    print("Installing torchani via uv ...")
    os.system("uv pip install --system torchani | cat")
    importlib.invalidate_caches()
    try:
        import torchani  # noqa: F401
        print("TorchANI ready.")
    except Exception as e:
        raise RuntimeError(f"TorchANI installation failed: {e}")

⠧ typing-extensions==4.15.0                                                     ⠋ Resolving dependencies...                                                     error: Failed to fetch: `https://pypi.org/simple/aiohttp/`
  Caused by: Request failed after 3 retries
  Caused by: error sending request for url (https://pypi.org/simple/aiohttp/)
  Caused by: operation timed out
Resolved 109 packages in 0.84ms
error: Distribution `gpu4pyscf-cuda12x==1.4.3 @ registry+https://pypi.org/simple` can't be installed because it doesn't have a source distribution or wheel for the current platform

hint: You're on Linux (`manylinux_2_36_aarch64`), but `gpu4pyscf-cuda12x` (v1.4.3) only has wheels for the following platforms: `manylinux_2_17_x86_64`, `manylinux2014_x86_64`; consider adding your platform to `tool.uv.required-environments` to ensure uv resolves to a version with compatible wheels


In [6]:
# (ersetzt durch uv-Installation in der vorherigen Zelle)


Installing torchani ...


/home/thomaspugh/projects/chem-properties/.venv/bin/python: No module named pip


ModuleNotFoundError: No module named 'torchani'

In [1]:
import os
import json
from typing import List, Tuple, Optional

import torch
import torchani
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdDistGeom

from data_gen_helpers import iterative_extend_smiles

# lade ANI2x
device = torch.device("cpu")
ani_model = torchani.models.ANI2x().to(device)
ani_model.eval()
print("Loaded ANI2x on", device)

xtb binary: None


OSError: xtb was not found. Please install and make it available in PATH. See docs: https://xtb-docs.readthedocs.io/en/latest/

In [ ]:
def remove_asterisk(smiles: str) -> str:
    return smiles.replace('*', '').replace('()', '')


def smiles_to_species_coords(smiles: str, add_hydrogen: bool = True, max_embed_attempts: int = 10):
    mol = Chem.MolFromSmiles(remove_asterisk(smiles))
    if mol is None:
        raise RuntimeError("Molecule failed to generate from smiles")

    if add_hydrogen:
        mol = Chem.AddHs(mol)

    success = False
    for i in range(max_embed_attempts):
        params = rdDistGeom.ETKDGv3()
        params.randomSeed = i * 777
        params.useRandomCoords = True
        params.maxAttempts = 1000
        code = rdDistGeom.EmbedMolecule(mol, params)
        if code == 0:
            success = True
            break

    if not success:
        for i in range(max_embed_attempts):
            code = rdDistGeom.EmbedMolecule(mol, randomSeed=i * 17)
            if code == 0:
                success = True
                break

    if not success or mol.GetNumConformers() == 0:
        raise RuntimeError("Failed to embed molecule geometry")

    conf = mol.GetConformer()
    species = []
    coords = []
    for atom in mol.GetAtoms():
        species.append(atom.GetSymbol())
        pos = conf.GetAtomPosition(atom.GetIdx())
        coords.append([pos.x, pos.y, pos.z])

    import numpy as np
    species = ''.join(species)
    coords = np.array([coords])
    return species, coords

In [ ]:
import numpy as np

def run_ani_singlepoint(species: str, coords) -> float:
    """Berechnet die Energie (Hartree) mit ANI2x bei gegebenen Species und Koordinaten."""
    # TorchANI expects species tensor and coordinates tensor
    converter = torchani.utils.ChemicalSymbolsToInts(ani_model.aev_computer.species)
    species_tensor = torch.tensor([converter(list(species))], device=device)
    coordinates = torch.tensor(coords, dtype=torch.float32, device=device)
    energy = ani_model((species_tensor, coordinates)).energies
    # energies shape: (batch,)
    return float(energy.item())


In [ ]:
import tempfile
from pathlib import Path

def run_xtb_singlepoint_xyz(xyz_block: str, charge: int = 0, uhf: int = 0, level: str = "GFN2-xTB", solvation: Optional[str] = None) -> Tuple[Optional[float], str]:
    """
    Runs an xTB single-point calculation for an XYZ (without header lines) and returns (energy in Eh, raw log).
    level: e.g. "GFN2-xTB", "GFN1-xTB" – mapped to --gfn 2/1.
    """
    if XTB_BIN is None:
        raise EnvironmentError("xtb binary not found")

    level_map = {"GFN2-xTB": "2", "GFN1-xTB": "1"}
    gfn = level_map.get(level, "2")

    with tempfile.TemporaryDirectory() as tmpd:
        tmp = Path(tmpd)
        xyz_path = tmp / "mol.xyz"
        # Write with header lines (number of atoms, comment) + coordinates
        coords = xyz_block.strip().splitlines()
        with xyz_path.open("w") as f:
            f.write(str(len(coords)) + "\n")
            f.write("generated by 05-xtb-calc-test\n")
            f.write("\n".join(coords) + "\n")

        cmd = [XTB_BIN, str(xyz_path), "--gfn", gfn, "--uhf", str(uhf), "--charge", str(charge)]
        if solvation:
            cmd += ["--alpb", solvation]

        try:
            proc = subprocess.run(cmd, cwd=tmp, capture_output=True, text=True, check=False)
            out = proc.stdout + "\n" + proc.stderr
        except Exception as e:
            return None, f"xTB call failed: {e}"

        energy = None
        # Search for total energy line in Eh
        # Example output (see xTB docs): "TOTAL ENERGY       -40.123456 Eh"
        for line in out.splitlines():
            ls = line.strip().lower()
            if "total energy" in ls and "eh" in ls:
                # robust extraction
                parts = line.replace("=", " ").replace(":", " ").split()
                vals = []
                for i, p in enumerate(parts):
                    if p.lower() in ("eh", "hartree") and i > 0:
                        try:
                            vals.append(float(parts[i-1]))
                        except:
                            pass
                if vals:
                    energy = vals[-1]
                    break
        return energy, out


In [ ]:
from multiprocessing import Pool, cpu_count, get_context
import time
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(processName)s - %(levelname)s - %(message)s')


def process_single_polymer_xtb(args) -> Tuple[str, str, Optional[float], str, float, int]:
    monomer_smiles, polymer_smiles, level, monomer_count = args
    start = time.time()
    try:
        xyz = smiles_to_xyz(polymer_smiles, add_hydrogen=True)
        energy, raw = run_xtb_singlepoint_xyz(xyz, charge=0, uhf=0, level=level)
        dur = time.time() - start
        return monomer_smiles, polymer_smiles, energy, level, dur, monomer_count
    except Exception as e:
        dur = time.time() - start
        logging.exception(f"Fehler fr {polymer_smiles}: {e}")
        return monomer_smiles, polymer_smiles, None, level+"_error", dur, monomer_count


def calculate_polymer_energies_xtb(monomer_smiles_list: List[str],
                                  max_chain_length: int = 60,
                                  level: str = "GFN2-xTB",
                                  max_polymers_per_monomer: Optional[int] = None,
                                  n_processes: Optional[int] = None) -> pd.DataFrame:
    if n_processes is None:
        n_processes = min(4, cpu_count())

    tasks = []
    for monomer in monomer_smiles_list:
        pairs = list(iterative_extend_smiles(monomer, max_chain_length, max_polymers_per_monomer))
        for polymer_smiles, monomer_count in pairs:
            tasks.append((monomer, polymer_smiles, level, monomer_count))

    results = []
    if not tasks:
        return pd.DataFrame(columns=["monomer_smiles", "polymer_smiles", "zero_energy", "method", "calc_time", "monomer_count"])    

    # Spawn-Kontext wie in opt_helpers
    with get_context("spawn").Pool(n_processes) as pool:
        for out in pool.imap_unordered(process_single_polymer_xtb, tasks):
            results.append(out)

    df = pd.DataFrame(results, columns=[
        'monomer_smiles', 'polymer_smiles', 'zero_energy', 'method', 'calc_time', 'monomer_count'
    ])
    df = df.dropna(subset=['zero_energy'])
    return df


In [ ]:
from multiprocessing import cpu_count, get_context
import time
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(processName)s - %(levelname)s - %(message)s')


def process_single_polymer_ani(args) -> Tuple[str, str, Optional[float], str, float, int]:
    monomer_smiles, polymer_smiles, level, monomer_count = args
    start = time.time()
    try:
        species, coords = smiles_to_species_coords(polymer_smiles, add_hydrogen=True)
        energy = run_ani_singlepoint(species, coords)
        dur = time.time() - start
        return monomer_smiles, polymer_smiles, energy, level, dur, monomer_count
    except Exception as e:
        dur = time.time() - start
        logging.exception(f"Fehler für {polymer_smiles}: {e}")
        return monomer_smiles, polymer_smiles, None, level+"_error", dur, monomer_count


def calculate_polymer_energies_ani(monomer_smiles_list: List[str],
                                  max_chain_length: int = 60,
                                  level: str = "ANI2x",
                                  max_polymers_per_monomer: Optional[int] = None,
                                  n_processes: Optional[int] = None) -> pd.DataFrame:
    if n_processes is None:
        n_processes = min(4, cpu_count())

    tasks = []
    for monomer in monomer_smiles_list:
        pairs = list(iterative_extend_smiles(monomer, max_chain_length, max_polymers_per_monomer))
        for polymer_smiles, monomer_count in pairs:
            tasks.append((monomer, polymer_smiles, level, monomer_count))

    results = []
    if not tasks:
        return pd.DataFrame(columns=["monomer_smiles", "polymer_smiles", "zero_energy", "method", "calc_time", "monomer_count"])    

    with get_context("spawn").Pool(n_processes) as pool:
        for out in pool.imap_unordered(process_single_polymer_ani, tasks):
            results.append(out)

    df = pd.DataFrame(results, columns=[
        'monomer_smiles', 'polymer_smiles', 'zero_energy', 'method', 'calc_time', 'monomer_count'
    ])
    df = df.dropna(subset=['zero_energy'])
    return df


In [ ]:
# Beispiel-Run: kleine Menge Monomere (ANI2x)

example_monomers = [
    "*C=C*",       # Ethylen-Fragment
    "*c1ccccc1*", # Phenyl-Fragment
]

res_df = calculate_polymer_energies_ani(
    example_monomers,
    max_chain_length=20,
    level="ANI2x",
    max_polymers_per_monomer=5,
    n_processes=min(2, cpu_count())
)

res_df.head()


In [ ]:
# Beispiel-Run: kleine Menge Monomere

example_monomers = [
    "*C=C*",       # Ethylen-Fragment
    "*c1ccccc1*", # Phenyl-Fragment
]

# Vorsicht: max_chain_length klein halten fr schnellen Test
res_df = calculate_polymer_energies_xtb(
    example_monomers,
    max_chain_length=20,
    level="GFN2-xTB",
    max_polymers_per_monomer=5,
    n_processes=min(2, cpu_count())
)

res_df.head()
